# 03 - Carrier Recovery (FPLL)

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from matplotlib import rcParams
rcParams['figure.figsize'] = (14, 5)
rcParams['figure.dpi'] = 100

from osmium.utils.constants import (
    TARGET_BASEBAND_RATE, ATSC_SYMBOL_RATE, ATSC_PILOT_FREQ,
)
from osmium.capture.adc_capture import ADCCapture
from osmium.demod.agc import AGC
from osmium.demod.dc_blocker import DCBlocker
from osmium.demod.fpll import FPLL

## 1. Load Baseband IQ

In [ ]:
BASEBAND_PATH = '../tests/fixtures/baseband_ch36.npz'

try:
    baseband, metadata = ADCCapture.load_from_file(BASEBAND_PATH)
    sample_rate = float(metadata.get('sample_rate', TARGET_BASEBAND_RATE))
    print(f'Loaded {len(baseband)} samples at {sample_rate/1e6:.4f} MSPS')
except FileNotFoundError:
    print('Baseband file not found. Generating synthetic ATSC-like signal...')
    sample_rate = TARGET_BASEBAND_RATE
    N = 200000
    t = np.arange(N) / sample_rate
    # Simulate carrier 
    carrier_offset = 150.0  
    pilot = 0.1 * np.ones(N)  
    data = np.random.choice([-7,-5,-3,-1,1,3,5,7], size=N).astype(np.float64)
    
    baseband = ((pilot + data) * np.exp(2j * np.pi * carrier_offset * t)).astype(np.complex64)
    baseband += 0.3 * (np.random.randn(N) + 1j * np.random.randn(N)).astype(np.complex64)
    metadata = {'sample_rate': sample_rate}

## 2. AGC + DC Blocker

In [ ]:
agc = AGC(alpha=1e-4, target_power=1.0)
dc = DCBlocker(alpha=1e-4)

x = agc.process(baseband)
x = dc.process(x)

print(f'AGC final gain: {agc.gain:.4f}')
print(f'Post-AGC mean power: {np.mean(np.abs(x)**2):.4f}')
print(f'DC offset (I): {np.mean(np.real(x)):.6f}')
print(f'DC offset (Q): {np.mean(np.imag(x)):.6f}')

## 3. FPLL Carrier Recovery

In [ ]:
%%time
fpll = FPLL(sample_rate=sample_rate, alpha=0.01)

BLOCK_SIZE = 5000
n_blocks = len(x) // BLOCK_SIZE

lock_history = []
freq_history = []
output_blocks = []

for i in range(n_blocks):
    block = x[i*BLOCK_SIZE:(i+1)*BLOCK_SIZE]
    out = fpll.process(block)
    output_blocks.append(out)
    lock_history.append(fpll.lock_indicator)
    freq_history.append(fpll.freq_offset_hz)

real_output = np.concatenate(output_blocks)

print(f'FPLL locked: {fpll.is_locked}')
print(f'Final lock indicator: {fpll.lock_indicator:.4f}')
print(f'Final freq offset: {fpll.freq_offset_hz:.2f} Hz')

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Lock acquisition curve
block_times = np.arange(len(lock_history)) * BLOCK_SIZE / sample_rate * 1e3
axes[0].plot(block_times, lock_history, 'b-', linewidth=1)
axes[0].axhline(y=0.8, color='r', linestyle='--', alpha=0.5, label='Lock threshold')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Lock Indicator')
axes[0].set_title('FPLL Lock Acquisition')
axes[0].set_ylim([-0.1, 1.1])
axes[0].grid(True, alpha=0.3)
axes[0].legend()

axes[1].plot(block_times, freq_history, 'g-', linewidth=1)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Frequency Offset (Hz)')
axes[1].set_title('FPLL Frequency Tracking')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Inspect Carrier-Recovered Output

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 8))

# Time domain
n_show = min(2000, len(real_output))
t_show = np.arange(n_show) / sample_rate * 1e6
axes[0,0].plot(t_show, real_output[:n_show], linewidth=0.5)
axes[0,0].set_xlabel('Time (us)')
axes[0,0].set_ylabel('Amplitude')
axes[0,0].set_title('Carrier-Recovered Output (time domain)')
axes[0,0].grid(True, alpha=0.3)

for lvl in [-7,-5,-3,-1,1,3,5,7]:
    axes[0,0].axhline(y=lvl, color='r', alpha=0.15, linewidth=0.5)

# Histogram 
settled = real_output[len(real_output)//2:]
axes[0,1].hist(settled, bins=200, density=True, alpha=0.7)
axes[0,1].set_xlabel('Amplitude')
axes[0,1].set_ylabel('Density')
axes[0,1].set_title('Amplitude Histogram (should show 8 peaks)')
axes[0,1].grid(True, alpha=0.3)

# Spectrum
nperseg = min(4096, len(real_output) // 4)
f_r, psd_r = welch(real_output, fs=sample_rate, nperseg=nperseg, scaling='density')
axes[1,0].plot(f_r/1e6, 10*np.log10(psd_r+1e-20), linewidth=0.5)
axes[1,0].set_xlabel('Frequency (MHz)')
axes[1,0].set_ylabel('PSD (dB/Hz)')
axes[1,0].set_title('Spectrum After Carrier Recovery')
axes[1,0].grid(True, alpha=0.3)

sps = 2
n_traces = 200
eye_len = 2 * sps
start = len(real_output) // 2
for i in range(n_traces):
    segment = real_output[start + i*sps : start + i*sps + eye_len]
    if len(segment) == eye_len:
        axes[1,1].plot(np.arange(eye_len), segment, 'b-', alpha=0.05, linewidth=0.5)
axes[1,1].set_xlabel('Sample')
axes[1,1].set_ylabel('Amplitude')
axes[1,1].set_title('Eye Diagram (2 samples/symbol, rough)')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Parameter Tuning

In [ ]:

alphas = [0.001, 0.005, 0.01, 0.02, 0.05]

fig, ax = plt.subplots(figsize=(12, 5))

for alpha in alphas:
    fpll_test = FPLL(sample_rate=sample_rate, alpha=alpha)
    lock_trace = []
    for i in range(n_blocks):
        block = x[i*BLOCK_SIZE:(i+1)*BLOCK_SIZE]
        _ = fpll_test.process(block)
        lock_trace.append(fpll_test.lock_indicator)
    ax.plot(block_times, lock_trace, label=f'alpha={alpha}')

ax.axhline(y=0.8, color='r', linestyle='--', alpha=0.5, label='Lock threshold')
ax.set_xlabel('Time (ms)')
ax.set_ylabel('Lock Indicator')
ax.set_title('FPLL Lock Acquisition vs Alpha')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Save Output for Next Stage

In [ ]:
CARRIER_RECOVERED_PATH = '../tests/fixtures/carrier_recovered_ch36.npz'

np.savez_compressed(
    CARRIER_RECOVERED_PATH,
    samples=real_output,
    meta_sample_rate=np.array(sample_rate),
    meta_channel=np.array(metadata.get('channel', 36)),
)
print(f'Saved {len(real_output)} carrier-recovered samples to {CARRIER_RECOVERED_PATH}')